In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42
)

In [ ]:
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
class FristApi(tf.keras.Model):
    def __init__(self, units=30, acti="relu", **kwagrs):
        super().__init__(**kwagrs)
        self.Norm_wide = tf.keras.layers.Normalization()
        self.Norm_deep = tf.keras.layers.Normalization()
        self.hidden_1 = tf.keras.layers.Dense(units, activation=acti)
        self.hidden_2 = tf.keras.layers.Dense(units, activation=acti)
        self.main_output = tf.keras.layers.Dense(1)
        self.aux_output = tf.keras.layers.Dense(1)

    def call(self, inputs):
        input_wide, input_deep = inputs
        norm_wide = self.Norm_wide(input_wide)
        norm_deep = self.Norm_deep(input_deep)
        hidden1 = self.hidden_1(norm_deep)
        hidden2 = self.hidden_2(hidden1)
        concat = tf.keras.layers.concatenate([norm_wide, hidden2])
        output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return output, aux_output


tf.random.set_seed(42)  # extra code – just for reproducibility
model = FristApi(30, acti="relu", name="my_cool_model")

In [ ]:
call_backs = tf.keras.callbacks.TensorBoard(log_dir="./logs", update_freq=1)

In [ ]:
class CustomEarlyStopping(tf.keras.callbacks.Callback):
    def __init__(self, ratio):
        super().__init__()
        self.ratio = ratio
        




In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(
    loss="mse",
    loss_weights=[0.9, 0.1],
    optimizer=optimizer,
    metrics=["RootMeanSquaredError"],
)
model.Norm_wide.adapt(X_train_wide)
model.Norm_deep.adapt(X_train_deep)


In [ ]:
history = model.fit(
    (X_train_wide, X_train_deep),
    (y_train, y_train),
    epochs=10,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid)),
    callbacks=[call_backs],
)
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results
y_pred_main, y_pred_aux = model.predict((X_new_wide, X_new_deep))

In [ ]:
history.history.keys()

In [ ]:
model.save("my_keras_model", save_format="tf")